## Evaluation metric for TSDR

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["font.size"] = 7
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['axes.linewidth'] = 1.0
plt.rcParams['axes.grid'] = True

In [3]:
import neptune

In [5]:
# https://docs.neptune.ai/api/project/#fetch_runs_table

project = neptune.init_project(
    project="yuuk1/tsdr",
    mode="read-only",
)
runs_table_df = project.fetch_runs_table(state="inactive").to_pandas()

https://app.neptune.ai/yuuk1/tsdr/


,sys/creation_time,sys/description,sys/failed,sys/hostname,sys/id,sys/modification_time,sys/monitoring_time,sys/name,sys/owner,sys/ping_time,...,monitoring/fa6a5674/memory,monitoring/fa6a5674/pid,monitoring/fa6a5674/tid,monitoring/fe610dbf/cpu,monitoring/fe610dbf/hostname,monitoring/fe610dbf/memory,monitoring/fe610dbf/pid,monitoring/fe610dbf/stderr,monitoring/fe610dbf/stdout,monitoring/fe610dbf/tid
0,2023-09-18 20:58:58.241000+00:00,,False,remotedev02,TSDR-1655,2023-09-18 21:05:10.027000+00:00,372,Untitled,yuuk1,2023-09-18 21:05:10.027000+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-09-18 20:55:57.739000+00:00,,False,remotedev02,TSDR-1654,2023-09-18 20:58:57.725000+00:00,180,Untitled,yuuk1,2023-09-18 20:58:57.725000+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-09-18 20:49:50.298000+00:00,,False,remotedev02,TSDR-1653,2023-09-18 20:55:57.179000+00:00,367,Untitled,yuuk1,2023-09-18 20:55:57.179000+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-09-18 20:46:51.614000+00:00,,False,remotedev02,TSDR-1652,2023-09-18 20:49:49.777000+00:00,178,Untitled,yuuk1,2023-09-18 20:49:49.777000+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-09-18 20:40:50.073000+00:00,,False,remotedev02,TSDR-1651,2023-09-18 20:46:51.061000+00:00,361,Untitled,yuuk1,2023-09-18 20:46:51.061000+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2023-09-18 13:12:25.095000+00:00,,False,remotedev02,TSDR-1588,2023-09-18 13:14:54.621000+00:00,149,Untitled,yuuk1,2023-09-18 13:14:54.621000+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,2023-09-18 13:09:55.860000+00:00,,False,remotedev02,TSDR-1587,2023-09-18 13:12:24.559000+00:00,149,Untitled,yuuk1,2023-09-18 13:12:24.559000+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,2023-09-18 13:07:26.623000+00:00,,False,remotedev02,TSDR-1586,2023-09-18 13:09:55.325000+00:00,149,Untitled,yuuk1,2023-09-18 13:09:55.325000+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,2023-09-18 13:04:52.962000+00:00,,False,remotedev02,TSDR-1585,2023-09-18 13:07:26.069000+00:00,153,Untitled,yuuk1,2023-09-18 13:07:26.069000+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
target_df = runs_table_df[runs_table_df["sys/failed"] == False].sort_values(by="sys/creation_time")[runs_table_df["experiment_id"].isin(["20230918231712", "20230918220219"])].filter(regex="^sys/id|experiment_id|dataset/.+|parameters/.*|scores/.*").dropna(axis=1, how="all")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(target_df)

/tmp/ipykernel_1431570/1788420304.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  target_df = runs_table_df[runs_table_df["sys/failed"] == False].sort_values(by="sys/creation_time")[runs_table_df["experiment_id"].isin(["20230918231712", "20230918220219"])].filter(regex="^sys/id|experiment_id|dataset/.+|parameters/.*|scores/.*").dropna(axis=1, how="all")


,sys/id,dataset/dataset_id,dataset/metric_types/containers,dataset/metric_types/middlewares,dataset/metric_types/nodes,dataset/metric_types/services,dataset/sampling_scale_factor,dataset/target_app,dataset/time_range/end,dataset/time_range/start,dataset/use_manually_selected_metrics,experiment_id,parameters/enable_multireducer,parameters/enable_unireducer,parameters/step1_changepoint_cost_model,parameters/step1_changepoint_penalty,parameters/step1_changepoint_search_method,parameters/step1_method_name,parameters/step2_changepoint_kde_bandwidth,parameters/step2_changepoint_kde_bandwidth_adjust,parameters/step2_changepoint_segment_selection_method,parameters/step2_clustering_granularity,parameters/step2_clustering_method_name,scores/RS_AC_1,scores/RS_AC_2,scores/RS_AC_3,scores/RS_AC_4,scores/RS_AC_5,scores/RS_AVG_1,scores/RS_AVG_2,scores/RS_AVG_3,scores/RS_AVG_4,scores/RS_AVG_5,scores/accuracy,scores/cause_metrics/num_found_mean,scores/cause_metrics/num_mandatory_found_mean,scores/cause_metrics/num_mandatory_total_mean,scores/cause_metrics/num_total_mean,scores/cause_metrics/proportion_mandatory_mean,scores/cause_metrics/proportion_mean,scores/cause_metrics/recall_mandatory_mean,scores/cause_metrics/recall_mean,scores/elapsed_time,scores/elapsed_time_max,scores/elapsed_time_min,scores/fn,scores/mandatory_accuracy,scores/mandatory_fn,scores/mandatory_tp,scores/num_series/containers,scores/num_series/middlewares,scores/num_series/services,scores/num_series/total,scores/reduction_rate_max,scores/reduction_rate_mean,scores/reduction_rate_min,scores/tp
71,TSDR-1584,9n6mf,True,False,False,True,1.0,sock-shop,0.0,0.0,True,20230918220219,True,True,l2,bic,binseg,changepoint,scott,0.01,weighted_max,service,changepoint-kde,0.021221,0.041862,0.061939,0.081471,0.100474,0.021221,0.031541,0.041674,0.051623,0.061393,0.9375,1.6625,0.8625,1.0000,2.00,0.021221,0.039752,0.862500,0.831250,0.836668,1.1622,0.7471,5.0,0.8625,11.0,69.0,0/0/65,NaN,0/0/15,43/64/80,0.796875,0.318741,0.061538,75.0
70,TSDR-1585,9n6mf,True,False,False,True,1.0,sock-shop,0.0,0.0,True,20230918220219,True,True,l2,bic,binseg,changepoint,scott,0.02,weighted_max,service,changepoint-kde,0.023324,0.045934,0.067856,0.089113,0.109729,0.023324,0.034629,0.045705,0.056557,0.067191,0.9625,1.7000,0.8750,1.0000,2.00,0.023324,0.043758,0.875000,0.850000,0.860880,1.2608,0.7647,3.0,0.8750,10.0,70.0,0/0/65,NaN,0/0/15,42/64/80,0.796875,0.344285,0.061538,77.0
69,TSDR-1586,9n6mf,True,False,False,True,1.0,sock-shop,0.0,0.0,True,20230918220219,True,True,l2,bic,binseg,changepoint,scott,0.04,weighted_max,service,changepoint-kde,0.023492,0.046229,0.068241,0.089554,0.110193,0.023492,0.034861,0.045987,0.056879,0.067542,0.9500,1.6750,0.8500,1.0000,2.00,0.023492,0.044795,0.850000,0.837500,0.836307,1.1524,0.7330,4.0,0.8500,12.0,68.0,0/0/65,NaN,0/0/15,41/64/80,0.765625,0.360504,0.061538,76.0
68,TSDR-1587,9n6mf,True,False,False,True,1.0,sock-shop,0.0,0.0,True,20230918220219,True,True,l2,bic,binseg,changepoint,scott,0.06,weighted_max,service,changepoint-kde,0.023619,0.046475,0.068594,0.090005,0.110734,0.023619,0.035047,0.046229,0.057173,0.067885,0.9500,1.6750,0.8500,1.0000,2.00,0.023619,0.045036,0.850000,0.837500,0.829299,1.2641,0.7424,4.0,0.8500,12.0,68.0,0/0/65,NaN,0/0/15,41/64/80,0.765625,0.363770,0.076923,76.0
67,TSDR-1588,9n6mf,True,False,False,True,1.0,sock-shop,0.0,0.0,True,20230918220219,True,True,l2,bic,binseg,changepoint,scott,0.08,weighted_max,service,changepoint-kde,0.023647,0.046527,0.068668,0.090099,0.110846,0.023647,0.035087,0.046281,0.057236,0.067958,0.9500,1.6750,0.8500,1.0000,2.00,0.023647,0.044971,0.850000,0.837500,0.844560,1.1634,0.7541,4.0,0.8500,12.0,68.0,0/0/65,NaN,0/0/15,41/64/80,0.765625,0.365115,0.092308,76.0
66,TSDR-1589,9n6mf,True,False,False,True,1.0,sock-shop,0.0,0.0,True,20230918220219,True,True,l2,bic,binseg,changepoint,scott,0.10,weighted_max,service,changepoint-kde,0.023926,0.047053,0.069413,0.091036,0.111950,0.023926,0.035489,0.046797,0.057857,0.068675,0.9500,1.6625,0.8500,1.000

In [78]:
def convert_to_dataset_name(x: pd.Series) -> str:
    match x["dataset/target_app"], x["dataset/metric_types/middlewares"], x["dataset/use_manually_selected_metrics"]:
        case "sock-shop", _, True:
            return "SS-small"
        case "sock-shop", False, False:
            return "SS-medium"
        case "sock-shop", True, False:
            return "SS-large"
        case "train-ticket", _, True:
            return "TT-small"
        case "train-ticket", False, False:
            return "TT-medium"
        case "train-ticket", True, False:
            return "TT-large"
        case _:
            raise ValueError(f"Unknown combination:")


def confusion_matrix(x: pd.Series) -> tuple:
    num_total = int(x["scores/num_series/total"].split("/")[1])
    num_tn_fn = num_total - int(x["scores/num_series/total"].split("/")[0])
    num_positives = x["scores/cause_metrics/num_mandatory_total_mean"]
    num_negatives = num_total - num_positives
    num_tp = x["scores/cause_metrics/num_mandatory_found_mean"]
    num_fn = num_positives - num_tp
    num_tn = num_tn_fn - num_fn
    num_fp = num_negatives - num_tn
    return (num_tp, num_fp, num_fn, num_tn)

def balanced_accuracy(x: pd.Series) -> float:
    recall = x["scores/cause_metrics/recall_mandatory_mean"]
    tp, fp, fn, tn = confusion_matrix(x)
    selectivity = tn / (tn + fp)
    return (recall + selectivity) / 2

def mcc(x: pd.Series) -> float:
    tp, fp, fn, tn = confusion_matrix(x)
    return tp * tn - fp * fn / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

target_df["dataset_name"] = target_df.apply(convert_to_dataset_name, axis=1)
target_df["scores/ba"] = target_df.apply(balanced_accuracy, axis=1)
target_df["scores/mcc"] = target_df.apply(mcc, axis=1)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    ba_df = target_df.rename(columns={
        "scores/ba": "ba",
        "scores/mcc": "mcc",
        "scores/reduction_rate_mean": "rrate",
        "scores/cause_metrics/recall_mandatory_mean": "recall",
        "scores/cause_metrics/proportion_mandatory_mean": "proportion",
    }).filter(
        regex="^(sys/id|dataset_name|ba|mcc|rrate|recall|proportion)$|search|adjust",
        axis=1,
    )
    display(ba_df)

0.8625 42.1375 0.13749999999999996 20.8625
0.875 41.125 0.125 21.875
0.85 40.15 0.15000000000000002 22.85
0.85 40.15 0.15000000000000002 22.85
0.85 40.15 0.15000000000000002 22.85
0.85 40.15 0.15000000000000002 22.85
0.9 42.1 0.09999999999999998 20.9
0.925 45.075 0.07499999999999996 17.925
0.9 44.1 0.09999999999999998 18.9
0.9125 41.0875 0.08750000000000002 21.9125
0.9125 41.0875 0.08750000000000002 21.9125
0.9125 41.0875 0.08750000000000002 21.9125
0.9125 42.0875 0.08750000000000002 20.9125
0.9125 42.0875 0.08750000000000002 20.9125
0.975 47.025 0.025000000000000022 15.975
0.9875 50.0125 0.012499999999999956 12.9875
0.85 40.15 0.15000000000000002 22.85
0.85 38.15 0.15000000000000002 24.85
0.875 38.125 0.125 24.875
0.9 40.1 0.09999999999999998 22.9
0.9 41.1 0.09999999999999998 21.9
0.9 41.1 0.09999999999999998 21.9
0.9375 47.0625 0.0625 15.9375
0.9625 51.0375 0.03749999999999998 11.9625
3.55 93.45 0.8875000000000002 87.1125
5.0875 563.9125 1.7124999999999995 776.2875
3.5625 90.4375 0.8

,sys/id,parameters/step1_changepoint_search_method,parameters/step2_changepoint_kde_bandwidth_adjust,proportion,recall,rrate,ba,dataset_name,mcc
71,TSDR-1584,binseg,0.01,0.021221,0.862500,0.318741,0.596825,SS-small,17.969615
70,TSDR-1585,binseg,0.02,0.023324,0.875000,0.344285,0.611111,SS-small,19.119319
69,TSDR-1586,binseg,0.04,0.023492,0.850000,0.360504,0.606349,SS-small,19.397791
68,TSDR-1587,binseg,0.06,0.023619,0.850000,0.363770,0.606349,SS-small,19.397791
67,TSDR-1588,binseg,0.08,0.023647,0.850000,0.365115,0.606349,SS-small,19.397791
66,TSDR-1589,binseg,0.10,0.023926,0.850000,0.364421,0.606349,SS-small,19.397791
65,TSDR-1590,binseg,0.50,0.023236,0.900000,0.331734,0.615873,SS-small,18.792349
64,TSDR-1591,binseg,1.00,0.021398,0.925000,0.283813,0.604762,SS-small,16.565823
63,TSDR-1592,pelt,0.01,0.021719,0.900000,0.302576,0.600000,SS-small,16.990999
62,TSDR-1593,pelt,0.02,0.024070,0.912500,0.334759,0.630159,SS-small,19.980255


In [69]:
ba_df[["rrate", "recall", "proportion"]].corrwith(ba_df["ba"])

rrate         0.643455
recall       -0.060932
proportion    0.277586
dtype: float64

In [79]:
ba_df.groupby(["dataset_name"]).apply(lambda x: x.sort_values(by="ba", ascending=False).head(10))

sys/id parameters/step1_changepoint_search_method  \
dataset_name                                                            
SS-large     26  TSDR-1629                                       pelt   
             24  TSDR-1631                                       pelt   
             22  TSDR-1633                                       pelt   
             20  TSDR-1635                                       pelt   
             42  TSDR-1613                                     binseg   
             40  TSDR-1615                                     binseg   
             38  TSDR-1617                                     binseg   
             36  TSDR-1619                                     binseg   
             4   TSDR-1651                                   bottomup   
             46  TSDR-1609                                     binseg   
SS-medium    13  TSDR-1642                                   bottomup   
             15  TSDR-1640                                   bottomup   
             11  TSDR-1644                                   bottomup   
             7   TSDR-1648                                   bottomup   
             9   TSDR-1646                                   bottomup   
             27  TSDR-1628                                       pelt   
             5   TSDR-1650                                   bottomup   
             25  TSDR-1630                                       pelt   
             23  TSDR-1632                                       pelt   
             29  TSDR-1626                                       pelt   
SS-small     53  TSDR-1602                                   bottomup   
             52  TSDR-1603                                   bottomup   
             62  TSDR-1593                                       pelt   
             60  TSDR-1595                                       pelt   
             61  TSDR-1594                                       pelt   
             51  TSDR-1604                                   bottomup   
             50  TSDR-1605                                   bottomup   
             58  TSDR-1597                                       pelt   
             54  TSDR-1601                                   bottomup   
             59  TSDR-1596                                       pelt   

                 parameters/step2_changepoint_kde_bandwidth_adjust  \
dataset_name                                                         
SS-large     26                                               0.04   
             24                                               0.06   
             22                                               0.08   
             20                                               0.10   
             42                                               0.04   
             40                                               0.06   
             38                                               0.08   
             36                                               0.10   
             4                                                0.10   
             46                                               0.01   
SS-medium    13                                               0.02   
             15                                               0.01   
             11                                               0.04   
             7                                                0.08   
             9                                                0.06   
             27                                               0.04   
             5                                                0.10   
             25                                               0.06   
             23                                               0.08   
             29                                               0.02   
SS-small     53                                               0.04   
             52                                               0.06   
   

## Corr with localization score

In [82]:
project = neptune.init_project(
    project="yuuk1/tsdr-localization",
    mode="read-only", 
)
loc_runs_table_df = project.fetch_runs_table(state="inactive").to_pandas().filter(regex="^sys/id|experiment_id|dataset/.+|parameters/.*|tsdr/parameters/.|scores/.*").dropna(axis=1, how="all")

https://app.neptune.ai/yuuk1/tsdr-localization/


In [ ]:
loc_target_df = runs_table_df[runs_table_df["sys/failed"] == False].sort_values(by="sys/creation_time")[runs_table_df["experiment_id"].isin(["20230918231712", "20230918220219"])].filter(regex="^sys/id|experiment_id|dataset/.+|parameters/.*|scores/.*").dropna(axis=1, how="all")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(loc_target_df)